In [115]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd

In [120]:
class IntrinsicValueCalculator():
    # Run Constructor
    def __init__(self) -> None:
        # Define the list of assets and financials
        self.assets = ["AAPL", "ABBV", "ABT", "ACN", "ADBE", "ADI", "ADP", "AMAT", "AMD", "AMGN", "AMZN", "AVGO", "AXP", "BA", "BAC", "BKNG", "BLK", "BRK-B", "BSX", "C", "CAT", "CB", "CI", "CMCSA", "COP", "COST", "CRM", "CSCO", "CVX", "DE", "DHR", "DIS", "ELV", "ETN", "FI", "GE", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "INTU", "ISRG", "JNJ", "JPM", "KLAC", "KO", "LIN", "LLY", "LMT", "LOW", "LRCX", "MA", "MCD", "MDLZ", "MDT", "META", "MMC", "MRK", "MS", "MSFT", "MU", "NEE", "NFLX", "NKE", "NOW", "NVDA", "ORCL", "PANW", "PEP", "PFE", "PG", "PGR", "PLD", "PM", "QCOM", "REGN", "RTX", "SBUX", "SCHW", "SNPS", "SPGI", "SYK", "T", "TJX", "TMO", "TMUS", "TSLA", "TXN", "UNH", "UNP", "UPS", "V", "VRTX", "VZ", "WFC", "WMT", "XOM"]
        self.functions = ['BALANCE_SHEET', 'CASH_FLOW', 'EARNINGS', 'INCOME_STATEMENT']

        # Read the closing prices, beta values and growth rates of the stocks
        self.df = pd.read_csv('/content/drive/MyDrive/Internship Project/Datasets/sp500_close_data.csv')
        self.df = self.df.set_index('Date')
        self.df.index = pd.to_datetime(self.df.index)

        self.beta_df = pd.read_csv('/content/drive/MyDrive/Internship Project/Datasets/beta_values_200_day_window.csv')
        self.beta_df = self.beta_df.set_index('Date')
        self.beta_df.index = pd.to_datetime(self.beta_df.index)

        self.growth_rates_df = pd.read_csv('/content/drive/MyDrive/Internship Project/Datasets/stock_growth_rates.csv')
        self.growth_rates_df = self.growth_rates_df.set_index('Date')
        self.growth_rates_df.index = pd.to_datetime(self.growth_rates_df.index)

        # Define the two dates used to calculate Intrinsic Value and No. of years used for projecting Future Cash Flows
        self.dates = [pd.Timestamp('2022-09-30 00:00:00'), pd.Timestamp('2023-09-30 00:00:00')]
        self.num_years = 5

        # TEMPORARY
        # self.beta_df['AAPL'].loc[self.dates[0]] = 1.213
        # self.growth_rates_df['AAPL'].loc[self.dates[0]] = 0.0991

        # Define a DataFrame to store the intrinsic values of the stocks
        self.intrinsic_value_df = pd.DataFrame(index = self.dates, columns = self.assets)

        # Define the Risk-Free Rate and Expected Market Returns for 2022 and 2023 (Test set years) and Perpetual Growth Rate
        self.rfr_emr_df = pd.DataFrame(index = self.dates, columns = ['rfr', 'emr'])
        self.rfr_emr_df['rfr'] = [0.0383, 0.0459] # Need to obtain full DataFrame
        self.rfr_emr_df['emr'] = [-0.1811, 0.2629] # Need to obtain full DataFrame
        self.perpetual_growth_rate = 0.02

        # Read the financials of each stock
        self.extra_info_tickers = {}
        for ticker in self.assets:
            self.extra_info_tickers[ticker] = {}

            self.extra_info_tickers[ticker]['earnings'] = pd.read_csv(f'/content/drive/MyDrive/sp500_annual/{ticker}_EARNINGS.csv')
            self.extra_info_tickers[ticker]['earnings'] = self.extra_info_tickers[ticker]['earnings'].set_index('fiscalDateEnding')
            self.extra_info_tickers[ticker]['earnings'].index = pd.to_datetime(self.extra_info_tickers[ticker]['earnings'].index)

            self.extra_info_tickers[ticker]['financials'] = pd.read_csv(f'/content/drive/MyDrive/sp500_annual/{ticker}_INCOME_STATEMENT.csv')
            self.extra_info_tickers[ticker]['financials'] = self.extra_info_tickers[ticker]['financials'].set_index('fiscalDateEnding')
            self.extra_info_tickers[ticker]['financials'].index = pd.to_datetime(self.extra_info_tickers[ticker]['financials'].index)

            self.extra_info_tickers[ticker]['balance_sheet'] = pd.read_csv(f'/content/drive/MyDrive/sp500_annual/{ticker}_BALANCE_SHEET.csv')
            self.extra_info_tickers[ticker]['balance_sheet'] = self.extra_info_tickers[ticker]['balance_sheet'].set_index('fiscalDateEnding')
            self.extra_info_tickers[ticker]['balance_sheet'].index = pd.to_datetime(self.extra_info_tickers[ticker]['balance_sheet'].index)

            self.extra_info_tickers[ticker]['cash_flow'] = pd.read_csv(f'/content/drive/MyDrive/sp500_annual/{ticker}_CASH_FLOW.csv')
            self.extra_info_tickers[ticker]['cash_flow'] = self.extra_info_tickers[ticker]['cash_flow'].set_index('fiscalDateEnding')
            self.extra_info_tickers[ticker]['cash_flow'].index = pd.to_datetime(self.extra_info_tickers[ticker]['cash_flow'].index)

    # Get the closest date from the closing prices DataFrame for stock value if given date is not present
    def get_closest_date(self, date, df):
        differences = [abs(ts - date) for ts in df.index]
        closest_index = differences.index(min(differences))
        return df.index[closest_index]

    # Calculate Historical Free Cash Flows
    def get_historical_free_cash_flow(self, date, cash_df):
        operating_cash_flow = cash_df['operatingCashflow'].loc[date:]
        capital_expenditures = cash_df['capitalExpenditures'].loc[date:]
        free_cash_flows = operating_cash_flow - capital_expenditures
        return list(free_cash_flows)[::-1]

    # Estimate Future Cash Flows
    def estimate_future_cash_flows(self, asset, save_date, free_cash_flows):
        const_growth_rate_est = self.growth_rates_df[asset].loc[save_date]
        future_cash_flows = []
        for i in range(self.num_years):
            curr = free_cash_flows[-1] * (1 + const_growth_rate_est)
            free_cash_flows.append(curr)
            future_cash_flows.append(curr)
        return future_cash_flows

    # Calculate Cost of Equity for WACC
    def get_cost_of_equity(self, asset, save_date):
        risk_free_rate = self.rfr_emr_df['rfr'].loc[save_date]
        if save_date not in self.beta_df.index: beta_date = self.get_closest_date(save_date, self.beta_df)
        else: beta_date = save_date
        beta = self.beta_df[asset].loc[beta_date]
        market_risk_premium = self.rfr_emr_df['emr'].loc[save_date] - risk_free_rate

        cost_of_equity = risk_free_rate + (beta * market_risk_premium)
        return cost_of_equity

    # Calculate Cost of Debt for WACC
    def get_cost_of_debt(self, date, balance_df, income_df):
        interest_expense = income_df['interestExpense'].loc[date]
        total_debt = balance_df['shortTermDebt'].loc[date] + balance_df['longTermDebt'].loc[date]

        cost_of_debt = interest_expense / total_debt

        income_tax_expense = income_df['incomeTaxExpense'].loc[date]
        earnings_before_tax = income_df['incomeBeforeTax'].loc[date]

        tax_rate = income_tax_expense / earnings_before_tax

        after_tax_cost_of_debt = cost_of_debt * (1 - tax_rate)
        return after_tax_cost_of_debt, tax_rate

    # Get Weights of Equity and Debt for WACC
    def get_weights_of_equity_and_debt(self, asset, date, stock_date, balance_df):
        share_price = self.df[asset].loc[stock_date]
        num_shares_outstanding = balance_df['commonStockSharesOutstanding'].loc[date]

        market_value_of_equity = share_price * num_shares_outstanding

        market_value_of_debt = balance_df['totalLiabilities'].loc[date] # To be checked later

        weight_of_equity = market_value_of_equity / (market_value_of_equity + market_value_of_debt)
        weight_of_debt = market_value_of_debt / (market_value_of_equity + market_value_of_debt)

        return weight_of_equity, weight_of_debt

    # Calculation of Weighted Average Cost of Capital (WACC)
    def calculate_wacc(self, asset, date, stock_date, save_date, balance_df, income_df):
        weight_of_equity, weight_of_debt = self.get_weights_of_equity_and_debt(asset, date, stock_date, balance_df)
        cost_of_equity = self.get_cost_of_equity(asset, save_date)
        cost_of_debt, tax_rate = self.get_cost_of_debt(date, balance_df, income_df)
        wacc = (weight_of_equity * cost_of_equity) + (weight_of_debt * cost_of_debt * (1 - tax_rate))
        return wacc

    # Calculate the Present Value of Future Cash Flows
    def get_pv_of_future_cash_flows(self, future_cash_flows, wacc):
        present_value_future_cash_flows = []

        for i, val in enumerate(future_cash_flows):
            pv = val / ((1 + wacc) ** (i + 1))
            present_value_future_cash_flows.append(pv)
        return present_value_future_cash_flows

    # Estimate the Discounted Terminal Value
    def estimate_discounted_terminal_value(self, future_cash_flows, wacc):
        tv = (future_cash_flows[-1] * (1 + self.perpetual_growth_rate)) / (wacc - self.perpetual_growth_rate)

        # Discount the Terminal Value to Present Value
        pv_tv = tv / ((1 + wacc) ** self.num_years)
        pv_tv
        return pv_tv

    # Calculate the Intrinsic Value & adjust for Outstanding Shares
    def calculate_intrinsic_value(self, asset, date, stock_date, save_date, balance_df, income_df, cash_df):
        free_cash_flows = self.get_historical_free_cash_flow(date, cash_df)
        future_cash_flows = self.estimate_future_cash_flows(asset, save_date, free_cash_flows)
        wacc = self.calculate_wacc(asset, date, stock_date, save_date, balance_df, income_df)
        present_value_future_cash_flows = self.get_pv_of_future_cash_flows(future_cash_flows, wacc)
        pv_tv = self.estimate_discounted_terminal_value(future_cash_flows, wacc)

        num_shares_outstanding = balance_df['commonStockSharesOutstanding'].loc[date]
        intrinsic_value_per_share = (sum(present_value_future_cash_flows) + pv_tv) / num_shares_outstanding
        return intrinsic_value_per_share

    # Main Function
    def main(self):
        for _ in self.dates:
            for asset in self.assets:
                print(f"Asset: {asset}")
                balance_df = self.extra_info_tickers[asset]['balance_sheet']
                income_df = self.extra_info_tickers[asset]['financials']
                cash_df = self.extra_info_tickers[asset]['cash_flow']

                if _ not in self.df.index:
                  stock_date = self.get_closest_date(_, self.df)
                else:
                  stock_date = _

                print(self.df[asset].loc[stock_date])

                if (_ not in balance_df.index) or (_ not in income_df.index) or (_ not in cash_df.index):
                  date = self.get_closest_date(_, balance_df)
                else:
                  date = _

                self.intrinsic_value_df[asset].loc[_] = self.calculate_intrinsic_value(asset, date, stock_date, _, balance_df, income_df, cash_df)

In [121]:
ivc2 = IntrinsicValueCalculator()

In [122]:
ivc2.main()

Asset: AAPL
138.1999969482422
Asset: ABBV
134.2100067138672
Asset: ABT
96.76000213623048
Asset: ACN
257.29998779296875
Asset: ADBE
275.20001220703125
Asset: ADI
139.33999633789062
Asset: ADP
226.19000244140625
Asset: AMAT
81.93000030517578
Asset: AMD
63.36000061035156
Asset: AMGN
225.3999938964844
Asset: AMZN
113.0
Asset: AVGO
444.010009765625
Asset: AXP
134.91000366210938
Asset: BA
121.08000183105467
Asset: BAC
30.200000762939453
Asset: BKNG
1643.2099609375
Asset: BLK
550.280029296875
Asset: BRK-B
267.0199890136719
Asset: BSX
38.72999954223633
Asset: C
41.66999816894531
Asset: CAT
164.0800018310547
Asset: CB
181.8800048828125
Asset: CI
277.4700012207031
Asset: CMCSA
29.32999992370605
Asset: COP
102.33999633789062
Asset: COST
472.2699890136719
Asset: CRM
143.83999633789062
Asset: CSCO
40.0
Asset: CVX
143.6699981689453
Asset: DE
333.8900146484375
Asset: DHR
228.98049926757807
Asset: DIS
94.33000183105467
Asset: ELV
454.239990234375
Asset: ETN
133.36000061035156
Asset: FI
93.569999694824

In [123]:
ivc2.intrinsic_value_df

,AAPL,ABBV,ABT,ACN,ADBE,ADI,ADP,AMAT,AMD,AMGN,...,TXN,UNH,UNP,UPS,V,VRTX,VZ,WFC,WMT,XOM
2022-09-30,-55.446031,-390.502285,-61.415165,-191.637415,-153.301707,-107.236575,-111.393805,-51.425015,NaN,-607.499341,...,-59.643819,-560.730319,-212.941504,-104.901603,-155.791456,NaN,-129.416001,NaN,-250.493391,-810.517502
2023-09-30,28.621975,238.56725,17.016432,72.519298,59.110109,25.932778,90.319871,34.757241,2.0157,201.78977,...,7.371552,583.052477,66.324089,34.194792,99.125344,NaN,84.73437,NaN,6.810014,39.55567
